# Transforma arquivos do Tabnet
Como usar:

* Acessar http://tabnet.datasus.gov.br/cgi/deftohtm.exe?sih/cnv/nrbr.def
* Escolher
    - Linha: Município
    - Coluna: Ano processamento
    - Conteúdo: Internações
    - Período disponível: Selecionar de Jan/2015 a Dez/2017
    - Lista Morb CID-10: Selecionar doença (ex: traumatismo intracraniano)
* Escolher opção "colunas separadas por ";""
* Clicar em Mostra
* Copiar dados a partir do header até a ultima linha (desconsiderando a linha "Total";xxx)
* Criar um arquivo .csv nessa pasta com o nome da doença em questão (ex: traumatismo.csv)
* Atualizar o nome do arquivo e da doença no código abaixo
* Rodar o código

Import libraries

In [93]:
import pandas as pd

Creates quantity file

In [94]:
doenca = 'DIABETES_MELLITUS' # Atualizar aqui
table = pd.read_csv('diabetes.csv', sep=';') # Atualizar aqui
table['MUNCOD'] = table['Município'].str.split(' ', expand=True,)[0]
table.drop('Município', axis=1, inplace=True)
table.drop('Total', axis=1, inplace=True)
table = table.replace("-", 0)
table.to_csv('Quantity/' + doenca + '_15_17.csv')

Creates rates file

In [95]:
table = pd.read_csv('Quantity/' + doenca + '_15_17.csv')
pop15=pd.read_csv('../util/Population/POPULATION_2015.csv', index_col=[0])
pop16=pd.read_csv('../util/Population/POPULATION_2016.csv', index_col=[0])
pop17=pd.read_csv('../util/Population/POPULATION_2017.csv', index_col=[0])

Merge with 2015

In [96]:
result = pd.merge(table, pop15, left_on="MUNCOD", right_on="MUNCOD", how="inner")
result['2015_RATE'] = result['2015']/result['POPULATION'] * 100000
result = result[["2015_RATE", "2016", "2017", "MUNCOD"]]
result

,2015_RATE,2016,2017,MUNCOD
0,207.209321,33,33,110001
1,64.562410,6,14,110037
2,103.908956,12,15,110040
3,246.146633,23,20,110034
4,81.416845,114,151,110002
...,...,...,...,...
5511,200.653983,15,33,522200
5512,36.715212,1,6,522205
5513,36.396724,2,2,522220
5514,17.927573,2,4,522230


Merge with 2016

In [97]:
result = pd.merge(result, pop16, left_on="MUNCOD", right_on="MUNCOD", how="inner")
result['2016_RATE'] = result['2016']/result['POPULATION'] * 100000
result = result[["2015_RATE", "2016_RATE", "2017", "MUNCOD"]]
result

,2015_RATE,2016_RATE,2017,MUNCOD
0,207.209321,129.381322,33,110001
1,64.562410,42.878582,14,110037
2,103.908956,58.340221,15,110040
3,246.146633,136.078571,20,110034
4,81.416845,107.652791,151,110002
...,...,...,...,...
5511,200.653983,110.562394,33,522200
5512,36.715212,12.068549,6,522205
5513,36.396724,35.618878,2,522220
5514,17.927573,35.492458,4,522230


Merge with 2017

In [98]:
result = pd.merge(result, pop17, left_on="MUNCOD", right_on="MUNCOD", how="inner")
result['2017_RATE'] = result['2017']/result['POPULATION'] * 100000
result = result[["2015_RATE", "2016_RATE", "2017_RATE", "MUNCOD"]]
result

,2015_RATE,2016_RATE,2017_RATE,MUNCOD
0,207.209321,129.381322,129.732280,110001
1,64.562410,42.878582,99.679601,110037
2,103.908956,58.340221,71.715433,110040
3,246.146633,136.078571,119.424375,110034
4,81.416845,107.652791,140.667940,110002
...,...,...,...,...
5511,200.653983,110.562394,241.316271,522200
5512,36.715212,12.068549,71.454091,522205
5513,36.396724,35.618878,34.897924,522220
5514,17.927573,35.492458,70.298770,522230


Average rates

In [99]:
result["RATE"] = result[['2015_RATE', '2016_RATE', '2017_RATE']].mean(axis=1)
result = result[["RATE", "MUNCOD"]]
result.to_csv('Rates/' + doenca + "_RATE_15_17.csv")


